## Generating series scaled to local grid by elevation
First, do everything from generate-timeseries.ipynb (and catchment-full_series_generation.py).

Edit 12 Mar 2023: Use updated catchment means, force AR(4) for Kangerlussuaq, fewer realizations

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Generate example time series of SMB for a given catchment
Pre-select AR(1) for annual variability portion
Include noise from sparse covar method

Created on Mon Jun 21 11:54:52 2021

@author: lizz
"""

from sklearn.covariance import GraphicalLassoCV
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
from statsmodels.tsa.ar_model import AutoReg
import scipy.linalg
import glob


model_names = ['ANICE-ITM_Berends', 'CESM_kampenhout', 'dEBM_krebs','HIRHAM_mottram', 
                'NHM-SMAP_niwano', 'RACMO_noel', 'SNOWMODEL_liston']
highlight_model = 'ANICE-ITM_Berends'
highlight_catchment_name, highlight_catchment_id = 'KANGERLUSSUAQ', 101

## Read in time series
def read_catchment_series(fpath, anomaly=True):
    catchment_fpath = fpath
    catchment_tseries = pd.read_csv(catchment_fpath, index_col=0, parse_dates=[0])
    catchment_tseries.mask(catchment_tseries>1e30)
    anomaly_series = catchment_tseries - catchment_tseries.mean()
    if anomaly:
        return anomaly_series
    else:
        return catchment_tseries

def fit_catchment_series(tseries, which_model, comparison_n=range(1,6), 
                         seasonal=True):
    bic_per_n = pd.DataFrame(index=comparison_n, columns=model_names)
    
    if 'multi' in which_model:  ## allow multi-model mode reporting
        for m in model_names:
            for n in comparison_n:
                mod = AutoReg(tseries[m], n, trend='ct', seasonal=seasonal)
                results = mod.fit()
                bic_per_n[m][n] = results.bic
            bic_per_n[m] = pd.to_numeric(bic_per_n[m])
        best_n = bic_per_n.idxmin().mode()[0]
    else:
        for n in comparison_n:
            mod = AutoReg(tseries[which_model], n, trend='ct', seasonal=seasonal)
            results = mod.fit()
            bic_per_n[which_model][n] = results.bic
        bic_per_n[which_model] = pd.to_numeric(bic_per_n[which_model])
        best_n = bic_per_n[which_model].idxmin()
    
    bic_difference = bic_per_n.transform(lambda x: x-x.min())
    
    return best_n, bic_difference

def find_AR_residuals(tseries, which_model, chosen_n=1, 
                         seasonal=False):
    mod = AutoReg(tseries[which_model], chosen_n, trend='ct', seasonal=seasonal)
    results = mod.fit()
    resids = results.resid
    
    return resids

## Time series from AR(n) fits
mod_fits = {m: [] for m in model_names}
mods = {m: [] for m in model_names}

ctmt_fpath = glob.glob('/Users/lizz/Documents/GitHub/Data_unsynced/SMBMIP-processed/*-catchment_{}-tseries.csv'.format(highlight_catchment_id))[0]
s = read_catchment_series(ctmt_fpath, anomaly=True)
a = s.resample('A').sum()
best_n, _ = fit_catchment_series(a, which_model='multi', seasonal=False)
for m in model_names:
    mod = AutoReg(a[m], best_n, trend='ct', seasonal=False).fit()
    fv = mod.fittedvalues
    r = mod.resid
    mod_fits[m] = fv
    mods[m] = mod


## Residuals with spatial information
ar_resids = []
ts_toplot = []
for i in range(1, 200):
    # print(i)
    ctmt_fpath = glob.glob('/Users/lizz/Documents/GitHub/Data_unsynced/SMBMIP-processed/*-catchment_{}-tseries.csv'.format(i))[0]
    s = read_catchment_series(ctmt_fpath, anomaly=True)
    a = s.resample('A').sum()
    ts_toplot.append(a)
    r = find_AR_residuals(a, which_model=highlight_model, chosen_n=best_n, seasonal=False)
    ar_resids.append(r)

ar_resids -= np.mean(ar_resids, axis=0) # normalize
emp_C = np.corrcoef(ar_resids)

np.random.seed(0)
X = np.random.multivariate_normal(mean=np.zeros(len(ar_resids)), cov=emp_C, size=len(ar_resids[0]))

gl_model = GraphicalLassoCV()
gl_model.fit(X)
cov_ = gl_model.covariance_

## Now take the Cholesky decomposition of the sparse cov matrix and generate noise with it
L = scipy.linalg.cholesky(cov_, lower=True)
N = np.random.normal(size=np.shape(ar_resids)) # draws from normal dist

D = np.diag(np.std(ar_resids,1)) ## diagonal matrix of standard devs
scaled_noise = D @ L @ N


## Plot example time series of resids
## set matplotlib font size defaults
SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 16

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

noise_realizations = []
## Several realizations
for j in range(100):
    Nj = np.random.normal(size=np.shape(ar_resids))
    noise_j = D @ L @ Nj
    noise_realizations.append(noise_j[highlight_catchment_id-1])
noise_colors=cm.get_cmap('Blues')

## Plot a sum of the two
fig2, ax2 = plt.subplots(figsize=(10,6))
for k in range(len(noise_realizations)):
    ax2.plot(mods[highlight_model].predict(best_n, len(a)-1, dynamic=best_n)
             +noise_realizations[k], 
            color=noise_colors(k/len(noise_realizations)), alpha=0.5)
ax2.plot(ts_toplot[highlight_catchment_id-1][highlight_model], color='k',
         label='{} output'.format(highlight_model))
ax2.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax2.set(xlabel='Year', ylabel=r'Catchment SMB anomaly [mm w.e. a$^{-1}$]',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
ax2.legend(loc='best')
plt.show()

## Future forecast
yrs_after_1980 = 70
noise_into_future = []
for j in range(100):
    Nj = np.random.normal(size=(len(ar_resids), yrs_after_1980))
    noise_j = D @ L @ Nj
    noise_into_future.append(noise_j[highlight_catchment_id-1])

fig3, ax3 = plt.subplots(figsize=(10,6))
for k in range(len(noise_into_future)):
    ax3.plot(mods[highlight_model].predict(best_n, yrs_after_1980, dynamic=2*best_n)
             +noise_into_future[k][best_n-1::], 
            color=noise_colors(k/len(noise_realizations)), alpha=0.5)
ax3.plot(ts_toplot[highlight_catchment_id-1][highlight_model], color='k',
         label='{} output'.format(highlight_model))
ax3.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
# ax2.set(xlabel='Year', ylabel=r'Catchment SMB anomaly [mm w.e. a$^{-1}$]',
#         xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
#                 np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
#         xticklabels=(1980,1990,2000,2010))
ax3.legend(loc='best')
plt.show()


## Map plot
Read in Kangerlussuaq catchment outline, compute catchment mean, extract point values, and compute anomalies

In [ ]:
from shapely.geometry import MultiPoint, Polygon, Point
from shapely.ops import triangulate
from scipy.spatial import distance
import shapefile
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import pyproj as pyproj
from scipy import interpolate
import datetime
import time
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
###------------------------
### CHOOSE CATCHMENTS
###------------------------
catchments_to_pull = (101,)

###------------------------
### DATA READ-IN  AND PROJECTION
###------------------------

## Read in BedMachine grid to reproject SMB
gl_bed_path ='/Users/lizz/Documents/GitHub/Data_unsynced/BedMachine-Greenland/BedMachineGreenland-2017-09-20.nc'
fh = Dataset(gl_bed_path, mode='r')
xx = fh.variables['x'][:].copy() # x-coord (polar stereo (70, 45))
yy = fh.variables['y'][:].copy() # y-coord
ss = fh.variables['surface'][:].copy() # surface elevation
fh.close()

## Read in Mouginot catchments from shapefile
print('Reading in Mouginot catchments')
catchment_fn = '/Users/lizz/Documents/GitHub/Data_unsynced/Greenland-catchments-Mouginot/Greenland_Basins_PS_v1.4.2.'
sf = shapefile.Reader(catchment_fn) 

## Example SMB field read in for grid
print('Reading in example SMB field')
# nhm_smb_path = '/Users/lizz/Documents/GitHub/Data_unsynced/SMBMIP/NHM-SMAP_niwano-monthly-ERA-Interim-1980.nc'
nhm_smb_path = '/Volumes/Backup Plus/Large data moved 20220331/SMBMIP/NHM-SMAP_niwano-monthly-ERA-Interim-1980.nc'
fh2 = Dataset(nhm_smb_path, mode='r')
xlon_nhm = fh2.variables['LON'][:].copy() #x-coord (latlon)
ylat_nhm = fh2.variables['LAT'][:].copy() #y-coord (latlon)
fh2.close()

###------------------------
### SET UP SMB REPROJECTION
###------------------------

## Down-sample bed topo
x_3km = xx[::20] # sample at ~3 km resolution
y_3km = yy[::20]
s_3km = ss[::20,::20]

## Down-sample SMB
x_lon_h = xlon_nhm[::10, ::10] 
y_lat_h = ylat_nhm[::10, ::10] # resolution about 10 km

print('Creating reprojected meshgrid')
wgs84 = pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by SMB data
psn_gl = pyproj.Proj("+init=epsg:3413") # Polar Stereographic North used by BedMachine and Mankoff
xs, ys = pyproj.transform(wgs84, psn_gl, x_lon_h, y_lat_h)
Xmat, Ymat = np.meshgrid(x_3km, y_3km) # Downsampled BedMachine coords

In [ ]:
##------------------------
## CREATE FRAMEWORK
##------------------------

## Identify grid points within catchment
pts_all = [(xs.ravel()[k], ys.ravel()[k]) for k in range(len(xs.ravel()))]
pt_ctmts = {i: [] for i in catchments_to_pull}
for i in catchments_to_pull:
    print('Point-checking catchment {}'.format(sf.record(i)['NAME']))
    c = Polygon(sf.shape(i).points)
    pts_in = [Point(p).within(c) for p in pts_all]
    pts = np.asarray(pts_all)[pts_in]
    pt_ctmts[i] = pts

In [ ]:
area_for_rescale = c.area ## in projection coordinates, which should be sq m

rescaled_projections = {i: [] for i in range(len(noise_realizations))}
for k in range(len(noise_realizations)):
    pred = mods[highlight_model].predict(best_n, len(a)-1, dynamic=best_n)+noise_realizations[k]
    rp = np.array(pred)/area_for_rescale
    rescaled_projections[k] = rp

In [ ]:
rescaled_projections[0]

In [ ]:
df_rp = pd.DataFrame(rescaled_projections)

In [ ]:
df_rp

In [ ]:
## Create data frames to store per-model data
# model_names = ['ANICE-ITM_Berends', 'CESM_kampenhout', 'dEBM_krebs','HIRHAM_mottram', 
#                 'NHM-SMAP_niwano', 'RACMO_noel', 'SNOWMODEL_liston']
model_names = ['ANICE-ITM_Berends',]
years = range(1980,2010)
start_date = datetime.datetime(years[0],1,1)
end_date = datetime.datetime(years[-1],12,31)
dates = pd.date_range(start=start_date, end=end_date, freq='M')
example_df_per_ctchmnt = {i: 
                  {m: 
                   {n: 
                    {y: pd.DataFrame(columns=('elevation', 
                                                'point_smb'))  for y in years}
                    for n in range(12)}
                     for m in model_names}
                      for i in catchments_to_pull}

###------------------------
### EXTRACT EXAMPLE SMB FIELD FOR CATCHMENT
###------------------------

## Store regridded SMBs for use in each catchment to take all available data
smb_ex_monthly = {i: [] for i in range(12)}

for m in ('ANICE-ITM_Berends',):
    t0 = time.time()
    if m=='CESM_kampenhout':
        vname = 'SMBCORR'
    else:
        vname = 'SMBcorr'
    for y in years:
        ti = time.time()
#         fpath = '/Users/lizz/Documents/GitHub/Data_unsynced/SMBMIP/{}-monthly-ERA-Interim-{}.nc'.format(m, y)
        fpath = '/Volumes/Backup Plus/Large data moved 20220331/SMBMIP/{}-monthly-ERA-Interim-{}.nc'.format(m, y)
        fh = Dataset(fpath, mode='r')
        smb_m = fh.variables[vname][:].copy()
        fh.close()
        d_subset = [d for d in dates if d.year==y]
        for i in range(len(smb_m)): # for each month
            ## downsample SMB
            smb_ds = smb_m[i][::10, ::10]
            ## Sample SMB at each Delaunay triangle and sum
            for j in catchments_to_pull:
                points = pt_ctmts[j]
                elevations = []
                smb_point_vals = smb_ds.ravel()[pts_in] ## go back and revise this to use new pts_in for every catchment
                for p in points:
                    surf_x = (np.abs(x_3km - p[0])).argmin()
                    surf_y = (np.abs(y_3km - p[1])).argmin()
                    elevations.append(s_3km[surf_y, surf_x])
                example_df_per_ctchmnt[j][m][i][y] = example_df_per_ctchmnt[j][m][i][y].assign(elevation=elevations,
                                            point_smb=smb_point_vals)
                
        tf = time.time()
        print('Finished processing year {} in time {}s'.format(y, tf-ti))
    t1 = time.time()
    print('Finished processing model {} in time {}'.format(m, t1-t0))

In [ ]:
## Select points for tseries comparison
series_points = [
    (497.73, -2298.93), # near terminus of Kangerlussuaq, in km Easting, Northing
    (397.96, -2342.95), # small positive anomaly, southwest
    (483.06, -2161.00) # farther up in accumulation area
    ]
ser = {'Point 1':[], 'Point 2':[], 'Point 3':[]}
elev = []
for i,p in enumerate(series_points):
    idx = distance.cdist([1000*np.array(p)], pt_ctmts[101]).argmin()
    ts = []
    elev.append(example_df_per_ctchmnt[101]['ANICE-ITM_Berends'][1][1980]['elevation'][idx])
    for y in years:
        for m in range(12):
            d = example_df_per_ctchmnt[101]['ANICE-ITM_Berends'][m][y]['point_smb'][idx]
            ts.append(d)
    ser['Point {}'.format(i+1)] = ts
r = pd.date_range(start='1980', end='2010', freq='M') ## dates for plotting
df = pd.DataFrame(ser, index=r)

In [ ]:
## plot the series

fig, (ax1, ax2) = plt.subplots(2)
df.plot(ax=ax1)
ax1.set(ylabel='SMB anomaly [mm w.e.]', xlabel='Time')
df_rp.plot(ax=ax2)
plt.show()

## Scale by elevation

In [ ]:
elev

In [ ]:
from stochSMB import segments_fit

In [ ]:
df_c = example_df_per_ctchmnt[101]['ANICE-ITM_Berends']
dict_per_month = {i: [] for i in range(12)}
for i in range(12):
    d_to_join = [df_c[i][y] for y in years]
    dict_per_month[i] = pd.concat(d_to_join)

monthly_mbg = {i: [] for i in range(12)}
monthly_mult = {i: [] for i in range(12)}
for i in range(12):
    df_m = dict_per_month[i]
    pt_smbs = np.asarray(df_m.sort_values(by='elevation')['point_smb'])
    anomalies = pt_smbs - np.mean(pt_smbs)
    mult_factor = pt_smbs/np.mean(pt_smbs)
    px, py = segments_fit(np.asarray(df_m.sort_values(by='elevation')['elevation']),
                          # np.asarray(df.sort_values(by='elevation')['point_smb']),
                          anomalies,
                          maxcount=2)
    pxm, pym = segments_fit(np.asarray(df_m.sort_values(by='elevation')['elevation']),
                            # np.asarray(df.sort_values(by='elevation')['point_smb']),
                            mult_factor,
                            maxcount=2)
    monthly_mbg[i] = interpolate.interp1d(px,py)
    monthly_mult[i] = interpolate.interp1d(pxm, pym)
    

In [ ]:
monthly_correction = {'Point 1': [], 'Point 2':[], 'Point 3': []}
monthly_mult_correction = {'Point 1': [], 'Point 2':[], 'Point 3': []}
for j, el in enumerate(elev):
    mc = [monthly_mbg[i](el) for i in range(12)]
    mult_mc = [monthly_mult[i](el) for i in range(12)]
    monthly_correction['Point {}'.format(j+1)] = mc
    monthly_mult_correction['Point {}'.format(j+1)] = mult_mc
df_mc = pd.DataFrame(monthly_correction)
df_mc_mult = pd.DataFrame(monthly_mult_correction)

In [ ]:
df_mc_mult

In [ ]:
len(df_rp)

In [ ]:
monthly_preds_p1 = {r[j]: [] for j in range(len(df))} ## create one row for each date
monthly_preds_p2 = {r[j]: [] for j in range(len(df))} ## create one row for each date
monthly_preds_p3 = {r[j]: [] for j in range(len(df))} ## create one row for each date


count = 0
for k in range(len(df)):
    scale_factor_1 = df_mc['Point 1'][k%12]
    scale_factor_2 = df_mc['Point 2'][k%12]
    scale_factor_3 = df_mc['Point 3'][k%12]
    monthly_preds_p1[r[k]] = scale_factor_1 + np.array(df_rp.iloc[count])
    monthly_preds_p2[r[k]] = scale_factor_2 + np.array(df_rp.iloc[count])
    monthly_preds_p3[r[k]] = scale_factor_3 + np.array(df_rp.iloc[count])
    if k%12==11: # if we've reached the end of a year
        count+=1
    if count>28:
        break

Now let's try this again, using multiplicative factor instead.

In [ ]:
## No, let's not.  It doesn't work well.

# monthly_preds_p1 = {r[j]: [] for j in range(len(df))} ## create one row for each date
# monthly_preds_p2 = {r[j]: [] for j in range(len(df))} ## create one row for each date
# monthly_preds_p3 = {r[j]: [] for j in range(len(df))} ## create one row for each date


# count = 0
# for k in range(len(df)):
#     scale_factor_1 = df_mc_mult['Point 1'][k%12]
#     scale_factor_2 = df_mc_mult['Point 2'][k%12]
#     scale_factor_3 = df_mc_mult['Point 3'][k%12]
#     monthly_preds_p1[r[k]] = scale_factor_1 * np.array(df_rp.iloc[count])
#     monthly_preds_p2[r[k]] = scale_factor_2 * np.array(df_rp.iloc[count])
#     monthly_preds_p3[r[k]] = scale_factor_3 * np.array(df_rp.iloc[count])
#     if k%12==11: # if we've reached the end of a year
#         count+=1
#     if count>28:
#         break

In [ ]:
df_mpp1 = pd.DataFrame.from_dict(monthly_preds_p1, orient='index')
df_mpp2 = pd.DataFrame.from_dict(monthly_preds_p2, orient='index')
df_mpp3 = pd.DataFrame.from_dict(monthly_preds_p3, orient='index')

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(3, sharex=True)
for k in range(len(df_mpp1.columns)): ## plot all the realizations
    ax1.plot(df_mpp1[k], 
           color=noise_colors(k/len(df_mpp1.columns)), alpha=0.5) 
ax1.plot(df['Point 1'], color='k', lw=2.0) #plot the true local SMB from the process model
ax1.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax1.set(#xlabel='Year',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
ax1.legend(loc='best')

for k in range(len(df_mpp2.columns)): ## plot all the realizations
    ax2.plot(df_mpp2[k], 
           color=noise_colors(k/len(df_mpp2.columns)), alpha=0.5) 
ax2.plot(df['Point 2'], color='k', lw=2.0) #plot the true local SMB from the process model
ax2.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax2.set(#xlabel='Year', 
        ylabel=r'Local SMB [mm w.e. a$^{-1}$]',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
ax2.legend(loc='best')

for k in range(len(df_mpp3.columns)): ## plot all the realizations
    ax3.plot(df_mpp3[k], 
           color=noise_colors(k/len(df_mpp3.columns)), alpha=0.5) 
ax3.plot(df['Point 3'], color='k', lw=2.0) #plot the true local SMB from the process model
ax3.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax3.set(xlabel='Year',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
ax3.legend(loc='best')
plt.show()

In [ ]:
fig, ax = plt.subplots()
for k in range(len(df_mpp3.columns)): ## plot all the realizations
    ax.plot(df_mpp3[k], 
           color=noise_colors(k/len(df_mpp3.columns)), alpha=0.5) 
ax.plot(df['Point 3'], color='k', lw=2.0) #plot the true local SMB from the process model
ax.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax.set(xlabel='Year',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
ax.legend(loc='best')
plt.show()

fig, ax = plt.subplots()
ax.plot(df_rp)
plt.show()

## Revision 16 Nov 2022: plot only two points

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True)
for k in range(len(df_mpp1.columns)): ## plot all the realizations
    ax1.plot(df_mpp1[k], 
           color=noise_colors(k/len(df_mpp1.columns)), alpha=0.5) 
ax1.plot(df['Point 1'], color='k', lw=2.0) #plot the true local SMB from the process model
ax1.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax1.set(#xlabel='Year',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
# ax1.legend(loc='best')

for k in range(len(df_mpp2.columns)): ## plot all the realizations
    ax2.plot(df_mpp3[k], 
           color=noise_colors(k/len(df_mpp3.columns)), alpha=0.5) 
ax2.plot(df['Point 3'], color='k', lw=2.0, label='Process model output') #plot the true local SMB from the process model
ax2.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax2.set(xlabel='Year', 
#         ylabel=r'Local SMB [mm w.e. a$^{-1}$]',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
ax2.legend(loc='best')

# for k in range(len(df_mpp3.columns)): ## plot all the realizations
#     ax3.plot(df_mpp3[k], 
#            color=noise_colors(k/len(df_mpp3.columns)), alpha=0.5) 
# ax3.plot(df['Point 3'], color='k', lw=2.0) #plot the true local SMB from the process model
# ax3.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
# ax3.set(xlabel='Year',
#         xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
#                 np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
#         xticklabels=(1980,1990,2000,2010))
# ax3.legend(loc='best')
plt.show()

## Add map of where these points are

Single panel map with points highlighted by stars.  Colors on map could add more relevant info - how about the seasonal swing?  This would highlight how sub-annual variability is dominated by the near-terminus region.

In [ ]:
highlight_points = np.asarray([
    (497.73, -2298.93), # near terminus of Kangerlussuaq, in km Easting, Northing
#     (397.96, -2342.95), # small positive anomaly, southwest
    (483.06, -2161.00) # farther up in accumulation area
    ])

seasonal_swing = np.abs(example_df_per_ctchmnt[101]['ANICE-ITM_Berends'][11][1980]['point_smb'] - example_df_per_ctchmnt[101]['ANICE-ITM_Berends'][6][1980]['point_smb'])

fig, ax = plt.subplots(1, sharex=True, sharey=True)

example_smb = example_df_per_ctchmnt[101]['ANICE-ITM_Berends'][11][1980]['point_smb']
sc = ax.scatter(0.001*pt_ctmts[101][:,0], 0.001*pt_ctmts[101][:,1], 
            c=seasonal_swing,
                vmin = 0, vmax = 1000,
           cmap='Oranges')
ax.scatter(highlight_points[:,0], highlight_points[:,1], marker='*', s=10,
          color='b')
ax.set(aspect=1,
      xticks=(300, 400, 500), yticks=(-2100, -2200, -2300),
      )
cb = fig.colorbar(sc, ax=ax, extend='max', ticks=(0, 500, 1000))
plt.show()

Now flip the plotting order of the series so that they appear roughly corresponding with their map location (terminus lower on the page)

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, sharex=True, sharey=True)
for k in range(len(df_mpp2.columns)): ## plot all the realizations
    ax1.plot(df_mpp3[k], 
           color=noise_colors(k/len(df_mpp3.columns)), alpha=0.5) 
ax1.plot(df['Point 3'], color='k', lw=2.0, label='Process model output') #plot the true local SMB from the process model
ax1.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax1.set(xlabel='Year', 
#         ylabel=r'Local SMB [mm w.e. a$^{-1}$]',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
ax1.legend(loc='best')


for k in range(len(df_mpp1.columns)): ## plot all the realizations
    ax2.plot(df_mpp1[k], 
           color=noise_colors(k/len(df_mpp1.columns)), alpha=0.5) 
ax2.plot(df['Point 1'], color='k', lw=2.0) #plot the true local SMB from the process model
ax2.plot(np.NaN, np.NaN, color=noise_colors(0.5), label='Stochastic realizations')
ax2.set(#xlabel='Year',
        xticks=(np.datetime64('1980-01-01'), np.datetime64('1990-01-01'),
                np.datetime64('2000-01-01'), np.datetime64('2010-01-01')),
        xticklabels=(1980,1990,2000,2010))
# ax2.legend(loc='best')

plt.show()